# Assignment Data Structure and Solution
#### We can use this Jupyter Notebook to analysis the data structure of different file and variables to help us to solve these three model construction and analysis.

# First Class(CommandLine)
<big>**Calss Function**</big>:
   1. Judge the command line to accord with the stationary Formate.
   2. Print Help Inforamtion if the command formate is wrong.
   3. Default:<br>
      1)'index_nostoplist_nostemming.txt' / queries_nostoplist_nostemming.txt'. <br>
      2) For the First dataset(index?), it's the key vocabularly in the text. <br>
      3) For the second dataset(queries?), it's the summary of all the search queris. <br>
       
**Example Command Formate**:
```
> python ir_engine.py -o result.txt
```

In [13]:
#==============================================================================
# Importing
import sys, getopt, re
import time
#==============================================================================
# Command line processing
class CommandLine:
    def __init__(self):
        opts, args = getopt.getopt(sys.argv[1:], 'hspw:o:')
        opts = dict(opts)
        self.exit = True

        if '-h' in opts:
            self.printHelp()
            return

        if len(args) > 0:
            print("*** ERROR: no arg files - only options! ***", file=sys.stderr)
            self.printHelp()
            return

        if '-w' in opts:
            if opts['-w'] in ('binary', 'tf', 'tfidf'):
                self.termWeighting = opts['-w']
            else:
                warning = (
                    "*** ERROR: term weighting label (opt: -w LABEL)! ***\n"
                    "    -- value (%s) not recognised!\n"
                    "    -- must be one of: binary / tf / tfidf"
                    )  % (opts['-w'])
                print(warning, file=sys.stderr)
                self.printHelp()
                return
        else:
            self.termWeighting = 'binary'

        if '-o' in opts:
            self.outfile = opts['-o']
        else:
            print("*** ERROR: must specify output file (opt: -o FILE) ***",
                  file=sys.stderr)
            self.printHelp()
            return

        if '-s' in opts and '-p' in opts:
            self.indexFile   = 'index_withstoplist_withstemming.txt'
            self.queriesFile = 'queries_withstoplist_withstemming.txt'
        elif '-s' in opts:
            self.indexFile   = 'index_withstoplist_nostemming.txt'
            self.queriesFile = 'queries_withstoplist_nostemming.txt'
        elif '-p' in opts:
            self.indexFile   = 'index_nostoplist_withstemming.txt'
            self.queriesFile = 'queries_nostoplist_withstemming.txt'
        else:
            self.indexFile   = 'index_nostoplist_nostemming.txt'
            self.queriesFile = 'queries_nostoplist_nostemming.txt'
            
        self.exit = False

    def printHelp(self):
        progname = sys.argv[0]
        progname = progname.split('/')[-1] # strip off extended path
        help = __doc__.replace('<PROGNAME>', progname, 1)
        print(help, file=sys.stderr)


# Second Class(IndexLoader)
<big>**Calss Function**</big>:
   1. Use the **Regular Expression** to extract the information from original file and index in a new dictionary.
   2. The New Generator dictionary has 2 dimensions.

<big>**Data Structure(Python Dictionary)**</big>:<br>
   1. {(term name)->{(term name)-> Count}}

In [14]:
#==============================================================================
# Load (precomputed) Index File for (preprocessed) Document Collection
class IndexLoader:
    def __init__(self, indexFile):
        self.index = {}
        docidCountRE = re.compile('(\d+):(\d+)')
        f = open(indexFile, 'r')
        for line in f:
            term = line.split(' ', 1)[0]
            self.index[term] = {}
            for (docid, count) in docidCountRE.findall(line):
                docid = int(docid)
                self.index[term][docid] = int(count)
    def getIndex(self):
        return self.index

# Third Class(Queries)
<big>**Calss Function**</big>:
   1. Use the **Regular Expression** to extract the information from original file and index in a new dictionary.
   2. The New Generator dictionary has 2 or more dimensions.
   3. This class is to extract the queries and get the same dataset formate(python dictionary) as IndexLoader

<big>**Data Structure(Dictionary)**</big>:<br>
   1. {(qid name)->{(qid name)-> count_num}}

In [15]:
class Queries:
    def __init__(self, queriesFile):
        self.qStore = {}
        termCountRE = re.compile('(\w+):(\d+)')
        f = open(queriesFile, 'r')
        for line in f:
            qid = int(line.split(' ', 1)[0])
            self.qStore[qid] = {}
            for (term, count) in termCountRE.findall(line):
                self.qStore[qid][term] = int(count)
    
    def getQuery(self, qid):
        if qid in self.qStore:
            return self.qStore[qid]
        else:
            print("*** ERROR: unknown query identifier (\"%s\") ***" % qid, file=sys.stderr)
            if type(qid) == type(''):
                print('WARNING: query identifiers should be of type: integer', file=sys.stderr)
                print('         -- your query identifier is of type: string', file=sys.stderr)
            print(' -- program exiting', file=sys.stderr)
    
    def qids(self):
        return sorted(self.qStore)

# Forth Class(ResultStore)
<big>**Calss Function**</big>:<br>
<big>**Data Structure(Dictionary)**</big>:<br>

In [16]:
class ResultStore:
    def __init__(self, outfile):
        self.outfile = outfile
        self.results = []

    def store(self, qid, docids):
        if len(docids) > 10:
            docids = docids[:10]
        self.results.append((qid, docids))

    def output(self):
        with open(self.outfile, 'w') as out:
            for (qid, docids) in self.results:
                for docid in docids:
                    print(qid, docid, file=out)

# Target Class(Retrieve)

In [22]:
# TODO: Finish the Assignment Part
import math
import operator
class Retrieve:
    # Define all the usable class variables
    index = None
    termWeighting = None
    doc_len = dict()  # The dictionary to record all the documents' length
    tr_candidate = dict()  # Retrieval all the documents 
    tr_candidate_all = dict() # for every doc_id and count
    tr_candidate_id = list()  # Retrial all the documents ID and store for use binary model candidate
    tr_query_count = list()  # To store the query count/frequency

    # Create new Retrieve object storing index and termWeighting scheme
    def __init__(self, index, termWeighting):
        self.index = index
        self.termWeighting = termWeighting
        # get the length of the each documents and
        self.doc_length()

    # Firstly we should calculate the documents length by number of arisen vocabularly
    def doc_length(self):
        # Extract all the items from the index file and sum the arisen number
        tempor_list = self.index.items()
        for term,combination in tempor_list:
            for doc_id,show_count in combination.items():
                if doc_id in self.doc_len:
                    self.doc_len[doc_id] = self.doc_len[doc_id] + int(show_count)
                else:
                    self.doc_len[doc_id] = int(show_count)

    # Method performing retrieval for specified query
    def forQuery(self, query):
        # Compare the index and query
        self.getCandidate(index, query) 
        # Firstly we should judge the function the program operate
        if self.termWeighting is 'binary':
            output = self.binary_model(query)
        elif self.termWeighting is 'tf':
            output = self.tf_model(query)
        
        return output
        

    # Extract the query from the file and also generate a new corresponding dictionary
    # We delete all the irreverent values and integrate the dictionary into a list(Count all the count)
    def getCandidate(self, index, query):
        # Get the two list to store the term of query and number of key vocabularly
        queryIterm = list(query.keys())
        qtermCount = list(query.values())
        for each_iterm in set(queryIterm):            
            if each_iterm in index:   # If the target document includes the query key we should put the id into a document
                self.tr_candidate[each_iterm] = index[each_iterm] # get all the document data formate: doc_id:count
                for doc_id,count in index[each_iterm].items():
                    if doc_id in self.tr_candidate_all:
                        self.tr_candidate_all[doc_id][each_iterm] = count
                    else:
                        self.tr_candidate_all[doc_id] = {}
                id_list = list(index[each_iterm].keys())
                self.tr_candidate_id.extend(id_list)         

    def binary_model(self, query):
        file_len = self.doc_len
        Results = dict()
        # Calculate all the value from the self.tr_documentid and convert it into the set to improve the efficiency
        # We have built the tr_candidate_id to store all the show index file and number is the occurance of query     
        # We need to calculate the number of occurance of index in set of query
        binary_candidate = set(self.tr_candidate_id)
        score = 0        
        for doc_id in binary_candidate:
            for each_term,each_value in (query.items()):
                if each_term in self.tr_candidate_all[doc_id]:
                    score = score + self.tr_candidate_all[doc_id][each_term] * each_value
                else:
                    continue
            result = score / math.sqrt(self.doc_len[doc_id])
            Results[doc_id] = result
            score = 0
        # return value is a list value
        topscore = [doc_id for doc_id,value in sorted(Results.items(), key = operator.itemgetter(1), reverse = True)]
        return topscore
    
    def tf_model(self, query):
        # Generate two new data structure(dictionary) to store the terms and frequency 
        # First the datastructure is term->{doc_id -> frequncy}
        term_doc_frequency = {}
        Results = {}
        
        # Traversal all the queries and extract the frequency
        for term in query:
            if term in self.tr_candidate:
                for doc_id,count in self.tr_candidate[term].items():
                    term_frequency = 
    
              


<big>**Below Code is the Formate of Command(Please customize with on the basis of your requirements)**</big><br><br>

**How to generate command line As you wish**<br>

`sys.argv` is the python reserved variables and applied to `ArgumentParser.parse_args(args=None, namespace=None)` to be the one of the parameters. This is a list of strings, each of which is obviously derived from command line arguments, separated by spaces. We can redefine this list by below command.

We should delete the key parameters like `python` and put the remains into the list.

In [23]:
# Example Command Formate:
#  > python ir_engine.py -o result.txt
sys.argv = ['ir_engine.py', '-o', 'result.txt']

In [24]:
config = CommandLine()
index = IndexLoader(config.indexFile).getIndex()

In [25]:
retrieve = Retrieve(index, config.termWeighting)
queries = Queries(config.queriesFile)
allResults = ResultStore(config.outfile)

In [27]:
for i,y in queries.getQuery(1):
    print(i)
    print(y)

a
n


ValueError: too many values to unpack (expected 2)

In [56]:
for qid in queries.qids():
    query = queries.getQuery(qid)
    results = retrieve.forQuery(query)
    print(results)
    allResults.store(qid, results)

allResults.output()

[1591, 1938, 1827, 1033, 2319, 1523, 2379, 3087, 1393, 1605, 1680, 2371, 1264, 1885, 1173, 1519, 2188, 1657, 2380, 1440, 1544, 1071, 2344, 1226, 1626, 1542, 1315, 1161, 2629, 1571, 1844, 3048, 2948, 1304, 2218, 2398, 2153, 1937, 2357, 2003, 3023, 2541, 3127, 3112, 2841, 1959, 2740, 2126, 1665, 971, 2535, 2378, 2542, 637, 3068, 2501, 2424, 1518, 2560, 1108, 1179, 1482, 2054, 2065, 2984, 1505, 202, 2439, 1506, 1247, 2092, 1410, 1908, 2950, 2470, 1195, 2947, 2632, 1464, 1572, 972, 1956, 1170, 2955, 1238, 1030, 1719, 1589, 1295, 2311, 2274, 3131, 2288, 1902, 1685, 585, 2105, 1484, 1811, 2764, 2169, 3012, 284, 2786, 1744, 2878, 2219, 2951, 1752, 1682, 3082, 2177, 1697, 1412, 1829, 397, 3134, 2064, 1647, 1236, 1034, 3015, 252, 2819, 3137, 1225, 2703, 1032, 2917, 2500, 2567, 2526, 1502, 2667, 2849, 1347, 2800, 2811, 3153, 2370, 2959, 2681, 275, 2796, 671, 1824, 2859, 3151, 1487, 3049, 1962, 1989, 1525, 2747, 2201, 1533, 414, 2838, 2365, 2080, 3163, 2581, 594, 3091, 2865, 2928, 294, 1163, 2390

[1461, 3068, 1462, 2319, 2882, 1912, 2740, 3127, 2867, 2985, 2138, 1324, 2111, 1247, 3150, 1341, 2198, 2127, 2840, 2852, 1387, 1572, 3025, 322, 2479, 2372, 1071, 1591, 25, 2567, 2525, 1938, 1754, 2317, 2690, 1484, 2624, 1196, 2629, 2741, 1680, 1978, 1226, 2379, 2358, 18, 597, 652, 1360, 104, 3026, 3119, 855, 435, 1959, 2321, 651, 1996, 2405, 2578, 2796, 2995, 3136, 2314, 795, 2806, 3011, 2168, 1748, 1755, 2541, 2407, 2859, 1472, 2262, 1951, 2632, 1236, 238, 1069, 2583, 2818, 3072, 1930, 1541, 2345, 2738, 637, 143, 987, 1233, 3134, 1380, 2165, 3180, 1164, 2812, 3137, 1846, 1908, 239, 643, 2374, 2424, 1910, 1473, 1514, 1826, 2955, 1636, 2597, 2730, 1713, 1901, 2080, 92, 1489, 633, 1511, 2151, 1811, 3076, 2020, 2158, 1347, 3183, 2246, 2320, 2972, 417, 2536, 2095, 2888, 1848, 1267, 3006, 1464, 3078, 616, 2988, 1719, 1435, 2868, 2920, 59, 2236, 440, 144, 1050, 2950, 2283, 2406, 3050, 1552, 1845, 2798, 1097, 1728, 84, 1519, 2582, 3105, 1747, 3182, 2163, 2684, 1699, 2870, 1006, 1647, 1534, 18

[1725, 2198, 1196, 2798, 3011, 3000, 2864, 2297, 3015, 2985, 2358, 3068, 2859, 2632, 2669, 2181, 3153, 86, 1747, 2889, 3048, 1267, 2314, 1135, 1879, 1856, 1811, 59, 2882, 2867, 1650, 1572, 322, 2990, 2233, 1473, 84, 2002, 2319, 2827, 2625, 1750, 2738, 1997, 2178, 2902, 1727, 2740, 3135, 2916, 1951, 3050, 1366, 1892, 2978, 1728, 1632, 2138, 1677, 3092, 1912, 2194, 3078, 1834, 2746, 2344, 1748, 2154, 2494, 854, 2734, 3160, 1924, 2870, 2486, 2887, 1637, 3150, 2697, 3086, 1605, 1928, 2812, 1552, 2342, 851, 1726, 2911, 2165, 2481, 2928, 2721, 2918, 278, 1665, 1001, 2699, 2568, 1890, 2289, 3142, 2525, 2988, 2396, 1173, 1484, 104, 2127, 1705, 3134, 2851, 1325, 2167, 1098, 2188, 3090, 2541, 1003, 70, 1030, 3118, 3128, 1462, 536, 1694, 3080, 1570, 2197, 2886, 3148, 2950, 1534, 2989, 2106, 1925, 2597, 2046, 2479, 2424, 2095, 3077, 2195, 670, 1854, 2378, 1541, 1708, 2111, 2113, 2996, 1025, 2080, 2914, 615, 2499, 1709, 2020, 3087, 1844, 1976, 1518, 1134, 1527, 1543, 1029, 2931, 3049, 2326, 1404, 2

[2714, 278, 1397, 1366, 2554, 2002, 1386, 2167, 2916, 2344, 2198, 2229, 2181, 1847, 2931, 409, 2283, 3087, 371, 2827, 2950, 2233, 3071, 1665, 1474, 1696, 2709, 644, 2289, 671, 2734, 2314, 2864, 2874, 2178, 3160, 2884, 2060, 2165, 2031, 3153, 2358, 3000, 2470, 1525, 3077, 1267, 1750, 2746, 1658, 1725, 1827, 3069, 1135, 2177, 1396, 2889, 1628, 2800, 1854, 1879, 2572, 1650, 3015, 1834, 2646, 1928, 2046, 1961, 2484, 2166, 2342, 2883, 2979, 2940, 695, 1423, 2092, 2632, 1392, 2061, 717, 2968, 2987, 2721, 1393, 2699, 1677, 1747, 2389, 2164, 759, 2499, 1748, 1695, 3006, 1194, 1543, 3070, 2366, 1705, 1946, 2455, 1915, 2829, 1699, 464, 3092, 3121, 1098, 2401, 3049, 2525, 284, 1032, 2850, 2154, 2146, 1411, 3080, 2004, 2956, 3134, 1727, 1768, 1395, 319, 3086, 1976, 2809, 1724, 2094, 3199, 2623, 1570, 86, 2325, 2402, 2302, 1653, 1862, 2378, 2851, 1605, 2812, 2886, 2490, 2390, 1427, 2705, 3064, 2911, 2967, 2560, 2701, 2767, 1003, 1844, 1890, 1001, 2887, 1196, 724, 3128, 2216, 3125, 2645, 2697, 1358,

[1856, 2233, 1665, 1366, 1032, 1194, 2931, 2002, 2181, 2470, 1653, 2842, 2198, 329, 2344, 1727, 1108, 2572, 3048, 2186, 3160, 2554, 2990, 1457, 3092, 2911, 1543, 670, 3134, 3049, 464, 2265, 2568, 2195, 2092, 2390, 3076, 2154, 1671, 2020, 2197, 2734, 2883, 1381, 2289, 2721, 2753, 2812, 1847, 1003, 2302, 3090, 2886, 1827, 2358, 1695, 2113, 2188, 1206, 2889, 1637, 2916, 972, 2699, 2541, 671, 2389, 1652, 2829, 1680, 2164, 2342, 1719, 2922, 3015, 2216, 2864, 409, 2882, 1135, 1844, 585, 2264, 2217, 2645, 3087, 1664, 616, 796, 2202, 2598, 3077, 2870, 1961, 3020, 1099, 2846, 2940, 3182, 1939, 3002, 3013, 2046, 3153, 2163, 1236, 1534, 1805, 3025, 2167, 1807, 2003, 1747, 1855, 2746, 1455, 1572, 3035, 2893, 2632, 2697, 2402, 1267, 1503, 1705, 2437, 1415, 2851, 2669, 321, 1527, 1325, 1976, 695, 1699, 1412, 2314, 1748, 2196, 3028, 2622, 2354, 824, 1769, 1029, 2985, 2083, 1173, 3080, 1854, 1928, 1768, 3150, 40, 3117, 2542, 849, 2307, 724, 944, 1936, 2920, 3070, 1497, 1879, 1591, 1516, 2061, 86, 2165

[3077, 1665, 2219, 2864, 1856, 1997, 1695, 2931, 1003, 1844, 2746, 2484, 2344, 1727, 2378, 1423, 3015, 1543, 3050, 2061, 464, 3080, 527, 2597, 2494, 2178, 2181, 278, 3048, 3041, 3160, 1470, 536, 2650, 1012, 1527, 2233, 2916, 1961, 2451, 2846, 2020, 1939, 2886, 2002, 2227, 1135, 1885, 2341, 2645, 2883, 2470, 1936, 2739, 1491, 3049, 3135, 71, 3036, 2342, 2198, 2632, 2197, 615, 2682, 3002, 2031, 2829, 1188, 2390, 2167, 3140, 1811, 2202, 1421, 1307, 2568, 2546, 1650, 2554, 1366, 1747, 1169, 2925, 2264, 3027, 2106, 1029, 1788, 2186, 1425, 1433, 1931, 1892, 2396, 2402, 1934, 2876, 1834, 1408, 2033, 2113, 1854, 1572, 2990, 2289, 2610, 2889, 1847, 1108, 3138, 3153, 1196, 1497, 2572, 2377, 2922, 2138, 1032, 1591, 2956, 1638, 2046, 2977, 1829, 1879, 2083, 2622, 2184, 1534, 2870, 2721, 2937, 2195, 2851, 2958, 2920, 1974, 2105, 2051, 1741, 2828, 796, 1613, 367, 2700, 2499, 1705, 1469, 2203, 1726, 1976, 1725, 1724, 2188, 1807, 1170, 2421, 1906, 1928, 1550, 3133, 3082, 1395, 1327, 1325, 2842, 2232, 